In [1]:
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.1 MB/s eta 0:00:00a 0:00:01


In [7]:
!pip install --upgrade gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.1
    Uninstalling gdown-4.7.1:
      Successfully uninstalled gdown-4.7.1


In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
import gdown
gdown.download_folder(id="1L1dX7CmLUbFPxsp37z7suGEmMxudeDwX")

Retrieving folder contents


Processing file 1zevHeQSmjcxzXmf3hM2J5gmMkWO6ln4G .DS_Store
Retrieving folder 14aJya_B62tGaHTnUFZgZ3c1vlrXKg2IK .ipynb_checkpoints
Processing file 16nYJ28oHhn5TakEezE_sE2HTyJ-0hOpF EEG_loading-checkpoint.ipynb
Processing file 1rl6zcMoHwsCXR11vmYXvtB2d2wyJt_6F main.ipynb
Processing file 1krpg_1i1z7GkKmY5sfWxVZdeVVucSnNS person_test.npy
Processing file 1z2uz75YfVfrLJ0rIfCPUYGI6mr_Xqxx_ person_train_valid.npy
Processing file 1_I6GXMJDUveXoTPnlWRI2szoe5d7-YgE X_test.npy
Processing file 1Z4cNncLvLGM9NBXF1vqSnOFJUmAbBJgM X_train_valid.npy
Processing file 1levFOLOfv93OLosULZIFl8nhud-xZYm6 y_test.npy
Processing file 1dpTmZCTyRMidjwSI2aa7SE4ne2uEgS-w y_train_valid.npy


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1zevHeQSmjcxzXmf3hM2J5gmMkWO6ln4G
To: /home/project/.DS_Store
100%|██████████████████████████████████████████████████████████████████████████████████| 6.15k/6.15k [00:00<00:00, 662kB/s]
Downloading...
From: https://drive.google.com/uc?id=16nYJ28oHhn5TakEezE_sE2HTyJ-0hOpF
To: /home/project/.ipynb_checkpoints/EEG_loading-checkpoint.ipynb
100%|█████████████████████████████████████████████████████████████████████████████████| 1.74k/1.74k [00:00<00:00, 5.70MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rl6zcMoHwsCXR11vmYXvtB2d2wyJt_6F
To: /home/project/main.ipynb
100%|███████████████████████████████████████████████████████████████████████████████████| 214k/214k [00:00<00:00, 2.51MB/s]
Downloading...
From: https://drive.google.com/uc?id=1krpg_1i1z7GkKmY5sfWxVZdeVVucSnNS
To: /home/project/person_test.npy
100%|█████████████████

['/home/project/.DS_Store',
 '/home/project/.ipynb_checkpoints/EEG_loading-checkpoint.ipynb',
 '/home/project/main.ipynb',
 '/home/project/person_test.npy',
 '/home/project/person_train_valid.npy',
 '/home/project/X_test.npy',
 '/home/project/X_train_valid.npy',
 '/home/project/y_test.npy',
 '/home/project/y_train_valid.npy']

In [2]:
X_test = np.load("project/X_test.npy")  # (443, 22, 1000)
y_test = np.load("project/y_test.npy")  # (443, 4)  # (num_trials, output types) # one hot encoded
person_train_valid = np.load("project/person_train_valid.npy")  # (2115, 1)  vals from 0-8 for participant
X_train_valid = np.load("project/X_train_valid.npy")  # (2115, 22, 1000)
# print(X_train_valid.shape)  # (2115, 22, 1000)  # (num_trials, channels, time bins)x
y_train_valid = np.load("project/y_train_valid.npy")  # (2115,)
person_test = np.load("project/person_test.npy")  # (443, 1)

In [3]:
def train_data_prep(X,y,sub_sample,average,noise):

    total_X = None
    total_y = None

    # Trimming the data (sample,22,1000) -> (sample,22,800)
    X = X[:,:,0:800]
    print('Shape of X after trimming:',X.shape)

    # Maxpooling the data (sample,22,800) -> (sample,22,800/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)


    total_X = X_max
    total_y = y
    print('Shape of X after maxpooling:',total_X.shape)

    # Averaging + noise
    X_average = np.mean(X.reshape(X.shape[0], X.shape[1], -1, average),axis=3)
    X_average = X_average + np.random.normal(0.0, 0.5, X_average.shape)

    # Data augmentation: increases number of samples (max pool and mean pool +noise)
    total_X = np.vstack((total_X, X_average))
    total_y = np.hstack((total_y, y))
    print('Shape of X after averaging+noise and concatenating:',total_X.shape)

    # Subsampling

    for i in range(sub_sample):

        X_subsample = X[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)

        total_X = np.vstack((total_X, X_subsample))
        total_y = np.hstack((total_y, y))


    print('Shape of X after subsampling and concatenating:',total_X.shape)
    print('Shape of Y:',total_y.shape)
    return total_X,total_y


# Don't increase the number of test samples
def test_data_prep(X):

    total_X = None


    # Trimming the data (sample,22,1000) -> (sample,22,800)
    X = X[:,:,0:800]
    print('Shape of X after trimming:',X.shape)

    # Maxpooling the data (sample,22,800) -> (sample,22,800/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, 2), axis=3)


    total_X = X_max
    print('Shape of X after maxpooling:',total_X.shape)

    return total_X

In [4]:
# Train val split, then data augment the training set
ind_valid = np.random.choice(X_train_valid.shape[0], 1000, replace=False)
ind_train = np.array(list(set(range(X_train_valid.shape[0])).difference(set(ind_valid))))
# Creating the training and validation sets using the generated indices
(x_train_unprocessed, x_valid) = X_train_valid[ind_train], X_train_valid[ind_valid]
(y_train_unprocessed, y_valid) = y_train_valid[ind_train], y_train_valid[ind_valid]

x_train, y_train = train_data_prep(x_train_unprocessed, y_train_unprocessed,2,2,True)
x_valid, y_valid = train_data_prep(x_valid, y_valid, 2, 2, True)
X_test_prep = test_data_prep(X_test)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_valid.shape[2], 1)
x_test = X_test_prep.reshape(X_test_prep.shape[0], X_test_prep.shape[1], X_test_prep.shape[2], 1)
x_train = np.swapaxes(x_train, 1,3)
x_train = np.swapaxes(x_train, 1,2)
x_valid = np.swapaxes(x_valid, 1,3)
x_valid = np.swapaxes(x_valid, 1,2)
x_test = np.swapaxes(x_test, 1,3)
x_test = np.swapaxes(x_test, 1,2)

Shape of X after trimming: (1115, 22, 800)
Shape of X after maxpooling: (1115, 22, 400)
Shape of X after averaging+noise and concatenating: (2230, 22, 400)
Shape of X after subsampling and concatenating: (4460, 22, 400)
Shape of Y: (4460,)
Shape of X after trimming: (1000, 22, 800)
Shape of X after maxpooling: (1000, 22, 400)
Shape of X after averaging+noise and concatenating: (2000, 22, 400)
Shape of X after subsampling and concatenating: (4000, 22, 400)
Shape of Y: (4000,)
Shape of X after trimming: (443, 22, 800)
Shape of X after maxpooling: (443, 22, 400)


In [5]:
x_train = x_train.reshape(-1,400,22)
x_valid = x_valid.reshape(-1,400,22)
x_test = x_test.reshape(-1,400,22)

In [6]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [7]:
trainds = CustomDataset(x_train, y_train-769)
testds = CustomDataset(x_test, y_test-769)
validds = CustomDataset(x_valid, y_valid-769)

In [8]:
from torch.utils.data.dataloader import default_collate, DataLoader


device = torch.device('cuda:0')  # or whatever device/cpu you like

# the new collate function is quite generic
trainloader = DataLoader(trainds, batch_size=100, shuffle=True, 
                    collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))
testloader = DataLoader(testds, batch_size=100, shuffle=True, 
                    collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))
validloader = DataLoader(validds, batch_size=100, shuffle=True, 
                    collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))

In [48]:
from transformers import PatchTSMixerConfig, PatchTSMixerForTimeSeriesClassification

config = PatchTSMixerConfig(num_targets=4, context_length=400, num_input_channels=22)
model = PatchTSMixerForTimeSeriesClassification(config)

In [49]:
model = model.to("cuda:0")

In [50]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters())

In [51]:
for i in range(80):
    losses = []
    for x,y in trainloader:
        x = x.to(torch.float)
        y = y.to(torch.long)
        optimizer.zero_grad()
        yhat = model(x)[0]
        loss = criterion(yhat, y)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print(f"Loss {sum(losses)/len(losses)}")

Loss 1.3867128610610961
Loss 1.375259115960863
Loss 1.3593898190392388
Loss 1.3175734281539917
Loss 1.2632429997126262
Loss 1.1841843181186251
Loss 1.0924112677574158
Loss 0.981871849960751
Loss 0.8621355295181274
Loss 0.7446977112028333
Loss 0.6572926388846503
Loss 0.5630249268478817
Loss 0.4786636041270362
Loss 0.4162093844678667
Loss 0.36632757518026565
Loss 0.3167005148198869
Loss 0.28135286072889965
Loss 0.2384997116194831
Loss 0.21117081162002352
Loss 0.19790394256512325
Loss 0.16916111989153756
Loss 0.1554706742366155
Loss 0.15046215918329026
Loss 0.13974852826860215
Loss 0.137945752342542
Loss 0.11085463290413221
Loss 0.09187483675777912
Loss 0.09383437385161718
Loss 0.10832820944488049
Loss 0.10006634154253535
Loss 0.08400162677798007
Loss 0.0796561380641328
Loss 0.08492441231177913
Loss 0.07336901914742258
Loss 0.07899233305619822
Loss 0.0701573423213429
Loss 0.06509431170092689
Loss 0.059991248945395155
Loss 0.06615930282407337
Loss 0.050590035174455907
Loss 0.04968251627352

In [54]:
avgs = []
for x,y in testloader:
    x = x.to(torch.float)
    y = y.to(torch.long)
    yhat = model(x)[0]
    avgs.append((yhat.argmax(dim=-1)==y).sum().item()/x.shape[0])
print(sum(avgs) / len(avgs))

0.3064651162790698
